<a href="https://colab.research.google.com/github/jvkvasanth/Spark/blob/main/Spark_Query_Plan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.master("local[*]").appName("test").getOrCreate()
sc = sparkSession.sparkContext
sc.setLogLevel("INFO")

In [ ]:
df = sparkSession.read.parquet("Spark/data/data_skew/transactions.parquet")

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/content/Spark/data/data_skew/transactions.parquet.

In [ ]:
df.show(5,False)

+----------+----------+----------+---------------+----------+----+-----+---+-------------+------+-----------+
|cust_id   |start_date|end_date  |txn_id         |date      |year|month|day|expense_type |amt   |city       |
+----------+----------+----------+---------------+----------+----+-----+---+-------------+------+-----------+
|C0YDPQWPBJ|2010-07-01|2018-12-01|TZ5SMKZY9S03OQJ|2018-10-07|2018|10   |7  |Entertainment|10.42 |boston     |
|C0YDPQWPBJ|2010-07-01|2018-12-01|TYIAPPNU066CJ5R|2016-03-27|2016|3    |27 |Motor/Travel |44.34 |portland   |
|C0YDPQWPBJ|2010-07-01|2018-12-01|TETSXIK4BLXHJ6W|2011-04-11|2011|4    |11 |Entertainment|3.18  |chicago    |
|C0YDPQWPBJ|2010-07-01|2018-12-01|TQKL1QFJY3EM8LO|2018-02-22|2018|2    |22 |Groceries    |268.97|los_angeles|
|C0YDPQWPBJ|2010-07-01|2018-12-01|TYL6DFP09PPXMVB|2010-10-16|2010|10   |16 |Entertainment|2.66  |chicago    |
+----------+----------+----------+---------------+----------+----+-----+---+-------------+------+-----------+
only showi

In [ ]:
customer_df = sparkSession.read.parquet("Spark/data/data_skew/customers.parquet")
customer_df.show(5)

+----------+-------------+---+------+----------+-----+-----------+
|   cust_id|         name|age|gender|  birthday|  zip|       city|
+----------+-------------+---+------+----------+-----+-----------+
|C007YEYTX9| Aaron Abbott| 34|Female| 7/13/1991|97823|     boston|
|C00B971T1J| Aaron Austin| 37|Female|12/16/2004|30332|    chicago|
|C00WRSJF1Q| Aaron Barnes| 29|Female| 3/11/1977|23451|     denver|
|C01AZWQMF3|Aaron Barrett| 31|  Male|  7/9/1998|46613|los_angeles|
|C01BKUFRHA| Aaron Becker| 54|  Male|11/24/1979|40284|  san_diego|
+----------+-------------+---+------+----------+-----+-----------+
only showing top 5 rows



**Narrow** **Transoformation**

In [ ]:
customer_dataset = (customer_df
                    .filter(F.col("city") == "boston")
                    .withColumn("first_name", F.split(F.col("name"), " ")[0])
                    .withColumn("last_name", F.split(F.col("name"), " ")[1])
                    .drop("name")
                    .withColumnRenamed("first_name", "name")
                    .withColumnRenamed("last_name", "surname")
                    .select("cust_id", "name", "surname", "city")
                    )

customer_dataset.show(5)
customer_dataset.explain(True)


+----------+-----+--------+------+
|   cust_id| name| surname|  city|
+----------+-----+--------+------+
|C007YEYTX9|Aaron|  Abbott|boston|
|C08XAQUY73|Aaron| Lambert|boston|
|C094P1VXF9|Aaron| Lindsey|boston|
|C097SHE1EF|Aaron|   Lopez|boston|
|C0DTC6436T|Aaron|Schwartz|boston|
+----------+-----+--------+------+
only showing top 5 rows

== Parsed Logical Plan ==
'Project ['cust_id, 'name, 'surname, 'city]
+- Project [cust_id#82, age#84, gender#85, birthday#86, zip#87, city#88, name#270, last_name#252 AS surname#279]
   +- Project [cust_id#82, age#84, gender#85, birthday#86, zip#87, city#88, first_name#243 AS name#270, last_name#252]
      +- Project [cust_id#82, age#84, gender#85, birthday#86, zip#87, city#88, first_name#243, last_name#252]
         +- Project [cust_id#82, name#83, age#84, gender#85, birthday#86, zip#87, city#88, first_name#243, split(name#83,  , -1)[1] AS last_name#252]
            +- Project [cust_id#82, name#83, age#84, gender#85, birthday#86, zip#87, city#88, spli

In [ ]:
!git clone https://github.com/jvkvasanth/Spark.git

Cloning into 'Spark'...
remote: Enumerating objects: 362, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 362 (delta 2), reused 5 (delta 1), pack-reused 354 (from 2)
Receiving objects: 100% (362/362), 702.53 MiB | 23.21 MiB/s, done.
Resolving deltas: 100% (4/4), done.
Updating files: 100% (340/340), done.
